## Ансамбль стекинга. Финальное решение.

### Описание задания
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz

In [29]:
# Импортируем необходимые библиотеки

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import preprocessing

In [30]:
# Получаем данные по ссылке, сохраняем в переменной data

data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


In [31]:
# Приводим параметры к числовым значениям

data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1

In [32]:
# Создаем список параметров для обучения модели

columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

In [33]:
# Нормируем данные по отношению к среднему значению, сохраняем полученную структуру в переменной data_transformed

scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]
print(data_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float64(118), int64(1)
memory usage: 53.9 MB
None


In [34]:
# Определяем функцию оптимизации потребления памяти

def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [35]:
# оптимизируем потребление памяти

data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Построение базовых моделей.

In [36]:
# сохраняем структуру данных для обучения в переменной х

x = pd.DataFrame(data_transformed, columns=columns_transformed)

In [37]:
# CatBoost. Оптимальные параметры learning_rate=0.56, l2_leaf_reg=1, depth=6 были получены ранее методом grid_search()

model_catboost = CatBoostClassifier(iterations=100, learning_rate=0.56,
                random_seed=17, l2_leaf_reg=1, depth=6, loss_function="MultiClass",
                bootstrap_type="MVS", custom_metric="WKappa")
model_catboost.fit(x, data_transformed["Response"])

0:	learn: 1.5334059	total: 39.6ms	remaining: 3.92s
1:	learn: 1.4360528	total: 109ms	remaining: 5.35s
2:	learn: 1.3401902	total: 204ms	remaining: 6.61s
3:	learn: 1.3074440	total: 304ms	remaining: 7.3s
4:	learn: 1.2897825	total: 411ms	remaining: 7.81s
5:	learn: 1.2699052	total: 532ms	remaining: 8.33s
6:	learn: 1.2580956	total: 616ms	remaining: 8.19s
7:	learn: 1.2433300	total: 704ms	remaining: 8.09s
8:	learn: 1.2384292	total: 787ms	remaining: 7.95s
9:	learn: 1.2327742	total: 871ms	remaining: 7.84s
10:	learn: 1.2278178	total: 971ms	remaining: 7.86s
11:	learn: 1.2228948	total: 1.06s	remaining: 7.78s
12:	learn: 1.2177607	total: 1.16s	remaining: 7.76s
13:	learn: 1.2083812	total: 1.22s	remaining: 7.5s
14:	learn: 1.2065373	total: 1.31s	remaining: 7.44s
15:	learn: 1.2028399	total: 1.43s	remaining: 7.5s
16:	learn: 1.1987273	total: 1.53s	remaining: 7.49s
17:	learn: 1.1901642	total: 1.6s	remaining: 7.3s
18:	learn: 1.1882665	total: 1.71s	remaining: 7.3s
19:	learn: 1.1813592	total: 1.81s	remaining: 7

In [38]:
# GradientBoosting
model_GB = GradientBoostingClassifier(random_state=17, max_depth=14,
                max_features=27, min_samples_leaf=20, n_estimators=76)
model_GB.fit(x, data_transformed["Response"])

GradientBoostingClassifier(max_depth=14, max_features=27, min_samples_leaf=20,
                           n_estimators=76, random_state=17)

In [39]:
# XGBoost
model_XGB = XGBClassifier(max_depth=17, max_features=27,
                     n_estimators=76, min_samples_leaf=20, use_label_encoder=False)
model_XGB.fit(x, data_transformed["Response"])

[17:04:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_features, min_samples_leaf } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:04:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=17,
              max_features=27, min_child_weight=1, min_samples_leaf=20,
              missing=nan, monotone_constraints='()', n_estimators=76,
              n_jobs=12, num_parallel_tree=1, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [40]:
# LightGBM
model_LGBM = lgb.LGBMRegressor(random_state=17, max_depth=18,
                min_child_samples=19, num_leaves=34,
                objective="multiclass", num_class=8)
model_LGBM.fit(x, data_transformed["Response"])

LGBMRegressor(max_depth=18, min_child_samples=19, num_class=8, num_leaves=34,
              objective='multiclass', random_state=17)

In [41]:
# получаем по ссылке данные для тестирования и сохраняем их в переменной data_test

data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")


In [42]:
# приводим параметры к числовым значениям

data_test["Product_Info_2_1"] = data_test["Product_Info_2"].str.slice(0, 1)
data_test["Product_Info_2_2"] = pd.to_numeric(data_test["Product_Info_2"].str.slice(1, 2))
data_test.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data_test["Product_Info_2_1"].unique():
    data_test["Product_Info_2_1" + l] = data_test["Product_Info_2_1"].isin([l]).astype("int8")
data_test.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data_test.fillna(value=-1, inplace=True)


In [43]:
# оптимизируем потребление памяти для тестовых данных

data_test = reduce_mem_usage(data_test)
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                            columns=columns)))
print (data_test_transformed.info())

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float32(118)
memory usage: 8.9 MB
None


In [44]:
# создаем структуру для тестирования и сохраняем ее в переменной х_test

x_test = pd.DataFrame(data_test_transformed, columns=columns_transformed)

In [45]:
# оцениваем вероятность для каждого класса (Response от 1 до 8) для модели CatBoost
data_test_catboost_proba = pd.DataFrame(model_catboost.predict_proba(x_test))

In [46]:
# оцениваем вероятность для каждого класса (Response от 1 до 8) для модели GradientBoosting
data_test_GB_proba = pd.DataFrame(model_GB.predict_proba(x_test))

In [47]:
# оцениваем вероятность для каждого класса (Response от 1 до 8) для модели XGBoost
data_test_XGB_proba = pd.DataFrame(model_XGB.predict_proba(x_test))

In [48]:
# получаем предсказание для каждого класса (Response от 1 до 8) для модели LightGBM
data_test_LGBM = pd.DataFrame(model_LGBM.predict(x_test))

In [49]:
# создаем функцию для определения класса экспертным голосованием
def vote_class (x):
    a = np.argmax(x.values)
    return a

In [50]:
# переменная data_test_proba определяет максимальные вероятности предсказания класса среди моделей

clf_list = [data_test_catboost_proba, data_test_GB_proba, data_test_XGB_proba, data_test_LGBM]

data_test_proba = np.amax(clf_list, axis=0)

data_test_proba = pd.DataFrame(data_test_proba)

data_test_transformed["Response"] = data_test_proba.apply(vote_class, axis=1) + 1
print(data_test_transformed["Response"])

0        1
1        8
2        6
3        7
4        8
        ..
19760    8
19761    7
19762    6
19763    2
19764    4
Name: Response, Length: 19765, dtype: int64


In [51]:
# получаем по ссылке шаблон для отчета
submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
print (submission.head())

   Id  Response
0   1         8
1   3         8
2   4         8
3   9         8
4  12         8


In [52]:
# вводим результат в отчет
submission["Response"] = data_test_transformed["Response"].astype("int8")
print (submission.head())

   Id  Response
0   1         1
1   3         8
2   4         6
3   9         7
4  12         8


In [53]:
# сохраняем отчет в файле csv
submission.to_csv("submission.csv", index=False)